## **Conseguir los niveles del `código ATC`**

---
### **Librerías**

In [10]:
import json
import pandas as pd
import sys
import os
import re

# utils
# Agrega la ruta del directoriodonde está el utils al path de Python
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from utils import *

---
### **1. Cargar datos**

In [3]:
# Cargar el archivo Excel con los códigos ATC
atc_df = pd.read_excel("../../assets/Tabla_ATC_traducida.xlsx")

# Ver primeras filas del DataFrame
display(atc_df.head(3))

# Quitar columna "Unnamed: 0"
# atc_df = atc_df.drop(columns=["Unnamed: 0"])

# Ver primeras filas del DataFrame
display(atc_df.head(3))

,ATC code_L1,name_L1,ATC code_L2,name_L2,ATC code_L3,name_L3,ATC code_L4,name_L4,ATC code_L5,name_L5
0,A,sistema digestivo y metabolismo,A01,preparados estomatológicos,A01A,preparados estomatológicos,A01AA,agentes profilácticos para la caries,A01AA01,fluoruro de sodio
1,A,sistema digestivo y metabolismo,A01,preparados estomatológicos,A01A,preparados estomatológicos,A01AA,agentes profilácticos para la caries,A01AA02,monofluorofosfato de sodio
2,A,sistema digestivo y metabolismo,A01,preparados estomatológicos,A01A,preparados estomatológicos,A01AA,agentes profilácticos para la caries,A01AA03,olaflur


,ATC code_L1,name_L1,ATC code_L2,name_L2,ATC code_L3,name_L3,ATC code_L4,name_L4,ATC code_L5,name_L5
0,A,sistema digestivo y metabolismo,A01,preparados estomatológicos,A01A,preparados estomatológicos,A01AA,agentes profilácticos para la caries,A01AA01,fluoruro de sodio
1,A,sistema digestivo y metabolismo,A01,preparados estomatológicos,A01A,preparados estomatológicos,A01AA,agentes profilácticos para la caries,A01AA02,monofluorofosfato de sodio
2,A,sistema digestivo y metabolismo,A01,preparados estomatológicos,A01A,preparados estomatológicos,A01AA,agentes profilácticos para la caries,A01AA03,olaflur


---
### **2. Conseguir los `niveles ATC`**
Hacemos un join con la tabla `Tabla_ATC_traducida` (en assets) para conseguir los niveles del código ATC, en un nuevo json: `fichas_tecnicas_mapped_atc.json`

In [ ]:
import re


def extraer_nombre_medicamento(nombre_archivo):
    # Quitar la extensión, si la tiene
    nombre_sin_ext = nombre_archivo.replace(".txt", "")

    # Buscar la primera ocurrencia de dígitos (con posible coma o punto)
    # La búsqueda ignora mayúsculas/minúsculas (por si "MG" o "microgramos")
    patron = r"([\d]+(?:[.,]\d+)?)"
    match = re.search(patron, nombre_sin_ext)

    if match:
        # Se toma todo lo que está antes de donde empieza el dígito
        indice = match.start()
        # Extraemos la parte anterior y quitamos guiones bajos extra
        parte_nombre = nombre_sin_ext[:indice].rstrip("_")
    else:
        # Si no hay dígitos, tomamos las dos primeras palabras separadas por guion bajo
        partes = nombre_sin_ext.split("_")
        parte_nombre = " ".join(partes[:2])

    # Convertir guiones bajos a espacios (por si quedan) y a minúsculas
    resultado = parte_nombre.replace("_", " ").lower()
    return resultado


# Ejemplos para probar
ejemplos = [
    "PARACETAMOL_NORMON_650_mg_COMPRIMIDOS_EFG.txt",  # -> "paracetamol normon"
    "IBUPROFENO_THRIFEN_400_mg_TABLETAS.txt",  # -> "ibuprofeno thrifen"
    "AMOXICILINA_ACIDO_CLAVULANICO_875_mg_SUSPENSION.txt",  # -> "amoxicilina acido clavulanico"
    "METFORMINA_GLUCOPHAGE_850_mg_TABLETAS.txt",  # -> "metformina glucophage"
    "OROXELAM_7,5_MG_SOLUCION_BUCAL.txt",  # -> "oroxelam"
    "ABATTRA_100_MICROGRAMOS_HORA_PARCHES.txt",  # -> "abattra"
    "ABACAT_GRANULADO_PARA_SOLUCIÓN_ORAL.txt",  # -> "abacat granulado"
    "ZINDACLIN_1%_GEL.txt",  # -> "zindaclin"
]

for ejemplo in ejemplos:
    resultado = extraer_nombre_medicamento(ejemplo)
    print(f"{ejemplo} -> {resultado}")

# Ejemplo de asignación en un diccionario:
datos = {}
nombre_archivo = "PARACETAMOL_NORMON_650_mg_COMPRIMIDOS_EFG.txt"
datos["nombre_medicamento"] = extraer_nombre_medicamento(nombre_archivo)
print("\nEn el diccionario:", datos)

PARACETAMOL_NORMON_650_mg_COMPRIMIDOS_EFG.txt -> paracetamol normon
IBUPROFENO_THRIFEN_400_mg_TABLETAS.txt -> ibuprofeno thrifen
AMOXICILINA_ACIDO_CLAVULANICO_875_mg_SUSPENSION.txt -> amoxicilina acido clavulanico
METFORMINA_GLUCOPHAGE_850_mg_TABLETAS.txt -> metformina glucophage
OROXELAM_7,5_MG_SOLUCION_BUCAL.txt -> oroxelam
ABATTRA_100_MICROGRAMOS_HORA_PARCHES.txt -> abattra
ABACAT_GRANULADO_PARA_SOLUCIÓN_ORAL.txt -> abacat granulado
ZINDACLIN_1%_GEL.txt -> zindaclin

En el diccionario: {'nombre_medicamento': 'paracetamol normon'}


In [19]:
# Crear los diccionarios a partir del Excel
# Ajustamos los nombres de las columnas según la estructura proporcionada
grupo_anatomico_dict = {}  # Nivel (anatómico)
subgrupo_terapeutico_L2_dict = {}  # Nivel 2 (Subgrupo terapéutico)
subgrupo_terapeutico_L3_dict = {}  # Nivel 3 (Subgrupo terapéutico o farmacológico)
subgrupo_terapeutico_L4_dict = (
    {}
)  # Nivel 4 (Subgrupo terapéutico, farmacológico o químico)
principio_activo_dict = {}  # Nivel 5 (Principio activo)

# Mapear grupos anatómicos (nivel 1)
indice_nivel_1 = 0
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[indice_nivel_1], atc_df.columns[indice_nivel_1 + 1]]
).iterrows():
    if pd.notna(row[atc_df.columns[indice_nivel_1]]) and pd.notna(
        row[atc_df.columns[indice_nivel_1 + 1]]
    ):
        grupo_anatomico_dict[row[atc_df.columns[indice_nivel_1]]] = row[
            atc_df.columns[indice_nivel_1 + 1]
        ]

# Mapear subgrupos terapéuticos nivel 2
indice_nivel_2 = 2
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[indice_nivel_2], atc_df.columns[indice_nivel_2 + 1]]
).iterrows():
    if pd.notna(row[atc_df.columns[indice_nivel_2]]) and pd.notna(
        row[atc_df.columns[indice_nivel_2 + 1]]
    ):
        subgrupo_terapeutico_L2_dict[row[atc_df.columns[indice_nivel_2]]] = row[
            atc_df.columns[indice_nivel_2 + 1]
        ]

# Mapear subgrupos terapéuticos nivel 3
indice_nivel_3 = 4
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[indice_nivel_3], atc_df.columns[indice_nivel_3 + 1]]
).iterrows():
    if pd.notna(row[atc_df.columns[indice_nivel_3]]) and pd.notna(
        row[atc_df.columns[indice_nivel_3 + 1]]
    ):
        subgrupo_terapeutico_L3_dict[row[atc_df.columns[indice_nivel_3]]] = row[
            atc_df.columns[indice_nivel_3 + 1]
        ]

# Mapear subgrupos terapéuticos nivel 4
indice_nivel_4 = 6
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[indice_nivel_4], atc_df.columns[indice_nivel_4 + 1]]
).iterrows():
    if pd.notna(row[atc_df.columns[indice_nivel_4]]) and pd.notna(
        row[atc_df.columns[indice_nivel_4 + 1]]
    ):
        subgrupo_terapeutico_L4_dict[row[atc_df.columns[indice_nivel_4]]] = row[
            atc_df.columns[indice_nivel_4 + 1]
        ]

# Mapear principio activo nivel 5
indice_nivel_5 = 8
for _, row in atc_df.drop_duplicates(
    subset=[atc_df.columns[indice_nivel_5], atc_df.columns[indice_nivel_5 + 1]]
).iterrows():
    if pd.notna(row[atc_df.columns[indice_nivel_5]]) and pd.notna(
        row[atc_df.columns[indice_nivel_5 + 1]]
    ):
        principio_activo_dict[row[atc_df.columns[indice_nivel_5]]] = row[
            atc_df.columns[indice_nivel_5 + 1]
        ]


# Función para mapear el código ATC
def mapear_atc(ATC):
    if pd.notna(ATC) and isinstance(ATC, str):
        L1 = ATC[0] if len(ATC) >= 1 else None  # Nivel (anatómico)
        L2 = ATC[:3] if len(ATC) >= 3 else None  # Nivel 2 (Subgrupo terapéutico)
        L3 = (
            ATC[:4] if len(ATC) >= 4 else None
        )  # Nivel 3 (Subgrupo terapéutico o farmacológico)
        L4 = (
            ATC[:5] if len(ATC) >= 5 else None
        )  # Nivel 4 (Subgrupo terapéutico, farmacológico o químico)
        L5 = ATC[:7] if len(ATC) >= 7 else None  # Nivel 5 (Principio activo)

        grupo_anat = grupo_anatomico_dict.get(L1, None)
        subgr_ter_L2 = subgrupo_terapeutico_L2_dict.get(L2, None)
        subgr_ter_L3 = subgrupo_terapeutico_L3_dict.get(L3, None)
        subgr_ter_L4 = subgrupo_terapeutico_L4_dict.get(L4, None)
        principio_activo = principio_activo_dict.get(L5, None)

        return (
            L1,
            grupo_anat,
            L2,
            subgr_ter_L2,
            L3,
            subgr_ter_L3,
            L4,
            subgr_ter_L4,
            L5,
            principio_activo,
        )
    else:
        return (None, None, None, None, None, None, None, None, None, None)


# Cargar el archivo JSON con las fichas técnicas
with open(
    "../../data/outputs/1_data_acquisition/wrangler/medicamentos.json",
    "r",
    encoding="utf-8",
) as f:
    datos_json = json.load(f)

medicamentos_list = []
for nombre_archivo, datos in datos_json.items():
    datos["nombre_medicamento_completo"] = nombre_archivo.replace(".txt", "")
    datos["nombre_medicamento"] = extraer_nombre_medicamento(nombre_archivo)

    medicamentos_list.append(datos)

medicamentos_df = pd.DataFrame(medicamentos_list)
medicamentos_df = medicamentos_df.drop_duplicates()

if medicamentos_df["ATC"].isnull().any():
    print("Hay valores nulos en 'ATC', se van a llenar con 'None'.")
    medicamentos_df = medicamentos_df.fillna({"ATC": pd.NA})

mapeo_atc = medicamentos_df["ATC"].apply(mapear_atc)

medicamentos_df[
    [
        "ATC_Nivel_Anatomico",
        "Descripcion_Nivel_Anatomico",
        "ATC_Nivel_2_Subgrupo_Terapeutico",
        "Descripcion_Nivel_2_Subgrupo_Terapeutico",
        "ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico",
        "Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico",
        "ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico",
        "Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico",
        "ATC_Nivel_5_Principio_Activo",
        "Descripcion_Nivel_5_Principio_Activo",
    ]
] = pd.DataFrame(mapeo_atc.tolist(), index=medicamentos_df.index)

print("\nPrimeras filas del DataFrame final con mapeo de ATC:")
display(medicamentos_df.head())

medicamentos_df.to_json(
    "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json",
    orient="records",
    lines=False,
    force_ascii=False,
    indent=4,
)

print("\nDataFrame guardado correctamente en el archivo JSON.")

Hay valores nulos en 'ATC', se van a llenar con 'None'.

Primeras filas del DataFrame final con mapeo de ATC:


,indicaciones,posologia,contraindicaciones,advertencias,interacciones,fertilidad_embarazo,efectos_conducir,reacciones_adversas,sobredosis,ATC,...,ATC_Nivel_Anatomico,Descripcion_Nivel_Anatomico,ATC_Nivel_2_Subgrupo_Terapeutico,Descripcion_Nivel_2_Subgrupo_Terapeutico,ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico,Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico,ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico,Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico,ATC_Nivel_5_Principio_Activo,Descripcion_Nivel_5_Principio_Activo
0,en base a su efecto antiagregante plaquetario ...,posología: como inhibidor de la agregación pla...,no se debe administrar ácido acetilsalicílico ...,dado el efecto antiagregante plaquetario del á...,combinaciones contraindicadas : metotrexato ut...,embarazo el ácido acetilsalicílico atraviesa l...,no se ha observado ningún efecto sobre la capa...,los efectos adversos del ácido acetilsalicílic...,diagnóstico: - los síntomas de intoxicación mo...,B01AC06,...,B,sangre y órganos hematopoyéticos,B01,agentes antitrombóticos,B01A,agentes antitrombóticos,B01AC,inhibidores de la agregación plaquetaria excl....,B01AC06,ácido acetilsalicílico
1,tratamiento o alivio sintomático del dolor oca...,posología: dosis media recomendada: adultos y ...,no se debe administrar ácido acetilsalicílico ...,dado el efecto antiagregante plaquetario del á...,combinaciones contraindicadas : metotrexato ut...,embarazo el ácido acetilsalicílico atraviesa l...,no se ha observado ningún efecto sobre la capa...,los efectos adversos del ácido acetilsalicílic...,diagnóstico: - los síntomas de intoxicación mo...,N02BA01,...,N,sistema nervioso,N02,analgésicos,N02B,otros analgésicos y antipiréticos,N02BA,ácido salicílico y derivados,N02BA01,ácido acetilsalicílico
2,alivio de los síntomasde procesos catarrales y...,posología: adultos mayores de 18 años: 1 sobre...,insuficiencia renal y hepática grave. - hipers...,"- debido a su contenido en paracetamol, se deb...",interacciones debidas al paracetamol: el parac...,embarazo: paracetamol: datos epidemiológicos d...,este medicamento puede producir somnolencia al...,durante el periodo de utilización del paraceta...,paracetamol: la sintomatología por sobredosis ...,N02BE51,...,N,sistema nervioso,N02,analgésicos,N02B,otros analgésicos y antipiréticos,N02BE,anilidas,N02BE51,"paracetamol, combinaciones excl. psicolépticos"
3,abacavir accord está indicado en el tratamient...,abacavir se debe prescribir por médicos con ex...,hipersensibilidad a abacavir o a alguno de los...,reacciones de hipersensibilidad (ver también s...,el potencial de interacciones mediadas por el ...,"embarazo como norma general, cuando se tome la...",no se han realizado estudios de los efectos so...,en el caso de muchas reacciones adversas comun...,se han administrado dosis únicas de hasta 1.20...,J05AF06,...,J,antiinfecciosos para uso sistémico,J05,antivirales para uso sistémico,J05A,antivirales de acción directa,J05AF,inhibidores de transcriptasa inversa análogos ...,J05AF06,abacavir
4,abacavir/lamivudina accord está indicado en el...,el tratamiento debe ser prescrito por un médic...,hipersensibilidad a los principios activos o a...,se incluyen en este epígrafe las advertencias ...,este medicamento contiene abacavir y lamivudin...,"embarazo como norma general, cuando se decida ...",no se han realizado estudios de los efectos so...,resumen del perfil de seguridad las reacciones...,no se han identificado síntomas o signos espec...,J05AR02,...,J,antiinfecciosos para uso sistémico,J05,antivirales para uso sistémico,J05A,antivirales de acción directa,J05AR,"antivirales para tratamiento de VIH, combinaci...",J05AR02,lamivudina y abacavir



DataFrame guardado correctamente en el archivo JSON.


---
### **3. Comprobación final**

In [20]:
# Cargar el json_medicamentos.json
json_medicamentos = load_json_dict(
    "../../data/outputs/1_data_acquisition/wrangler/medicamentos.json",
)
fichas_tecnicas_mapped_atc = pd.read_json(
    "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json"
)

In [21]:
# Comparar el número de filas
num_filas_json_medicamentos = len(json_medicamentos)
num_filas_fichas_tecnicas_mapped_atc = len(fichas_tecnicas_mapped_atc)
if num_filas_json_medicamentos == num_filas_fichas_tecnicas_mapped_atc:
    print(
        f"El número de filas en json_medicamentos ({num_filas_json_medicamentos}) es igual al número de filas en fichas_tecnicas_mapped_atc ({num_filas_fichas_tecnicas_mapped_atc})."
    )
else:
    print(
        f"El número de filas en json_medicamentos ({num_filas_json_medicamentos}) es diferente al número de filas en fichas_tecnicas_mapped_atc ({num_filas_fichas_tecnicas_mapped_atc})."
    )

El número de filas en json_medicamentos (19909) es igual al número de filas en fichas_tecnicas_mapped_atc (19909).


In [22]:
# Contar nulos en cada campo de fichas_tecnicas_mapped_atc
nulos = fichas_tecnicas_mapped_atc.isnull().sum()
print("\nConteo de nulos en cada campo de fichas_tecnicas_mapped_atc:")
print(nulos)


Conteo de nulos en cada campo de fichas_tecnicas_mapped_atc:
indicaciones                                                       338
posologia                                                           81
contraindicaciones                                                  54
advertencias                                                        79
interacciones                                                      359
fertilidad_embarazo                                                203
efectos_conducir                                                    81
reacciones_adversas                                                 68
sobredosis                                                         143
ATC                                                               2317
Propiedades_farmacocineticas                                       339
excipientes                                                        368
incompatibilidades                                                 386
precauciones_co

In [23]:
# Ver los nulos en 'nombre_medicamento'
nulos_nombre_medicamento = fichas_tecnicas_mapped_atc[
    fichas_tecnicas_mapped_atc["nombre_medicamento"].isnull()
]
print("\nMedicamentos con nombre_medicamento nulo:")
display(nulos_nombre_medicamento["nombre_medicamento_completo"])


Medicamentos con nombre_medicamento nulo:


Series([], Name: nombre_medicamento_completo, dtype: object)